In [8]:
import argparse
import random

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [9]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from sampler import NewSampler

from DeepDSC.DeepDSC import (AE, DF, GeneExpressionDataset,
                             calculate_morgan_fingerprints, prepare_data,
                             prepare_drug_data, prepare_train_val_test_data,
                             train_autoencoder, train_df_model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
data = "nci"
PATH = "../nci_data/"

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
class Args:
    def __init__(self):
        self.device = device  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)
cells = {i: j for i, j in enumerate(res.index)}
drugs = {i: j for i, j in enumerate(res.columns)}

load nci


In [13]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

In [26]:
def main(PATH, train, val):
    print("Loading gene expression data...")
    normalized_gene_exp_tensor, gene_exp = prepare_data(
        data1=PATH + "/gene_exp_part1.csv.gz", data2=PATH + "gene_exp_part2.csv.gz"
    )
    normalized_gene_exp_dataset = GeneExpressionDataset(normalized_gene_exp_tensor)
    normalized_gene_exp_dataloader = DataLoader(
        normalized_gene_exp_dataset, batch_size=10000, shuffle=True
    )

    # オートエンコーダーのトレーニング
    print("Training autoencoder...")
    autoencoder = AE(normalized_gene_exp_tensor.shape[1]).to(device)
    train_autoencoder(autoencoder, normalized_gene_exp_dataloader)
    print("Autoencoder training completed.")

    # 圧縮特徴の抽出
    print("Extracting compressed features...")
    compressed_features_tensor = autoencoder.encoder(normalized_gene_exp_tensor)
    compressed_features = pd.DataFrame(
        compressed_features_tensor.cpu().detach().numpy(), index=gene_exp.columns
    )
    print(f"Compressed features shape: {compressed_features.shape}")
    drug_response, nsc_sm = prepare_drug_data(is_nsc=True)
    mfp = calculate_morgan_fingerprints(drug_response.T, nsc_sm)
    print(f"Morgan fingerprints shape: {mfp.shape}")

    train_labels = train[2]
    val_labels = val[2]
    train_data = train[[1, 0]]
    train_data.columns = [0, 1]
    val_data = val[[1, 0]]
    val_data.columns = [0, 1]
    print(
        f"Training data size: {len(train_data)}, Validation data size: {len(val_data)}"
    )
    train_data, val_data = prepare_train_val_test_data(
        train_data, val_data, compressed_features, mfp
    )
    df_model = DF().to(device)
    val_labels, best_val_out = train_df_model(
        df_model,
        train_data,
        val_data,
        torch.tensor(train_labels).double().to(device),
        torch.tensor(val_labels).double().to(device),
    )
    print("DF model training completed.")
    return val_labels, best_val_out

In [27]:
def DeepDSC(res_mat, null_mask, target_dim, target_index, seed):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)

    train_data = pd.DataFrame(sampler.train_data, index=res.index, columns=res.columns)
    test_data = pd.DataFrame(sampler.test_data, index=res.index, columns=res.columns)

    train_mask = pd.DataFrame(sampler.train_mask, index=res.index, columns=res.columns)
    test_mask = pd.DataFrame(sampler.test_mask, index=res.index, columns=res.columns)

    train = pd.DataFrame(train_mask.values.nonzero()).T
    train[2] = train_data.values[train_mask.values.nonzero()].astype(int)

    test = pd.DataFrame(test_mask.values.nonzero()).T
    test[2] = test_data.values[test_mask.values.nonzero()].astype(int)

    train[0] = [cells[i] for i in train[0]]
    train[1] = [drugs[i] for i in train[1]]
    test[0] = [cells[i] for i in test[0]]
    test[1] = [drugs[i] for i in test[1]]

    val_labels, best_val_out = main(PATH, train, test)
    return val_labels, best_val_out

In [28]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            val_labels, best_val_out = DeepDSC(
                res.values, null_mask.values, dim, target_index, seed
            )

    true_datas = pd.concat([true_datas, pd.DataFrame(val_labels.cpu().numpy())], axis=1)
    predict_datas = pd.concat(
        [predict_datas, pd.DataFrame(best_val_out.cpu().numpy())], axis=1
    )

  0%|          | 0/60 [00:00<?, ?it/s]/var/folders/y3/ssnk1ytd3m5bjmrchh2lt74srg76p8/T/ipykernel_77997/775812543.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if cell_sum[target_index] < 10:


Loading gene expression data...
Training autoencoder...



100%|██████████| 800/800 [05:13<00:00,  2.55it/s]


Autoencoder training completed.
Extracting compressed features...
Compressed features shape: (60, 500)
Morgan fingerprints shape: (1005, 256)
Training data size: 57035, Validation data size: 912
Epoch 1 Loss: 0.773 Val Loss: 2.133
Accuracy: 0.500
Epoch 2 Loss: 2.002 Val Loss: 1.471
Accuracy: 0.500
Epoch 3 Loss: 1.394 Val Loss: 0.747
Accuracy: 0.512
Epoch 4 Loss: 0.738 Val Loss: 0.803
Accuracy: 0.498
Epoch 5 Loss: 0.839 Val Loss: 0.922
Accuracy: 0.501
Epoch 6 Loss: 0.970 Val Loss: 0.849
Accuracy: 0.500
Epoch 7 Loss: 0.889 Val Loss: 0.724
Accuracy: 0.499
Epoch 8 Loss: 0.749 Val Loss: 0.682
Accuracy: 0.559
Epoch 9 Loss: 0.688 Val Loss: 0.730
Accuracy: 0.507
Epoch 10 Loss: 0.719 Val Loss: 0.787
Accuracy: 0.501
Epoch 11 Loss: 0.767 Val Loss: 0.801
Accuracy: 0.501
Epoch 12 Loss: 0.779 Val Loss: 0.773
Accuracy: 0.500
Epoch 13 Loss: 0.755 Val Loss: 0.728
Accuracy: 0.501
Epoch 14 Loss: 0.718 Val Loss: 0.692
Accuracy: 0.511
Epoch 15 Loss: 0.690 Val Loss: 0.675
Accuracy: 0.577
Epoch 16 Loss: 0.68

  2%|▏         | 1/60 [08:47<8:38:28, 527.26s/it]

...

Loading gene expression data...
Training autoencoder...



100%|██████████| 800/800 [06:29<00:00,  2.05it/s]


Autoencoder training completed.
Extracting compressed features...
Compressed features shape: (60, 500)
Morgan fingerprints shape: (1005, 256)
Training data size: 57079, Validation data size: 646
Epoch 1 Loss: 0.773 Val Loss: 2.155
Accuracy: 0.500
Epoch 2 Loss: 2.008 Val Loss: 1.496
Accuracy: 0.500
Epoch 3 Loss: 1.392 Val Loss: 0.768
Accuracy: 0.508
Epoch 4 Loss: 0.736 Val Loss: 0.824
Accuracy: 0.502
Epoch 5 Loss: 0.842 Val Loss: 0.941
Accuracy: 0.502
Epoch 6 Loss: 0.970 Val Loss: 0.867
Accuracy: 0.500
Epoch 7 Loss: 0.888 Val Loss: 0.745
Accuracy: 0.498
Epoch 8 Loss: 0.748 Val Loss: 0.705
Accuracy: 0.497
Epoch 9 Loss: 0.687 Val Loss: 0.755
Accuracy: 0.498
Epoch 10 Loss: 0.720 Val Loss: 0.813
Accuracy: 0.500
Epoch 11 Loss: 0.768 Val Loss: 0.827
Accuracy: 0.500
Epoch 12 Loss: 0.779 Val Loss: 0.799
Accuracy: 0.498
Epoch 13 Loss: 0.755 Val Loss: 0.754
Accuracy: 0.502
Epoch 14 Loss: 0.717 Val Loss: 0.718
Accuracy: 0.502
Epoch 15 Loss: 0.689 Val Loss: 0.702
Accuracy: 0.491
Epoch 16 Loss: 0.68

  7%|▋         | 4/60 [44:32<10:23:41, 668.23s/it]


KeyboardInterrupt: 

In [ ]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")